In [1]:
import QUANTAXIS as QA

# 在这里我们演示一下 下单/交易/结算的整个流程

我们首先会建立一个账户类和一个回测类


In [2]:
 # 初始化一个account
user= QA.QA_User(username='admin',password='940809x')
portfolio= user.new_portfolio('order_example')
Account=portfolio.new_account()

# 初始化一个回测类
B = QA.QA_BacktestBroker()


在第一天的时候,全仓买入 000001

In [3]:
# 全仓买入'000001'

Order=Account.send_order(code='000001',
                        price=11,
                        money=0.04*Account.cash_available,
                        time='2018-05-09',
                        towards=QA.ORDER_DIRECTION.BUY,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_MONEY
                        )




In [4]:

## 打印order的占用资金
print('ORDER的占用资金: {}'.format((Order.amount*Order.price)*(1+Account.commission_coeff)))

ORDER的占用资金: 39609.9


In [5]:
# 账户剩余资金
print('账户剩余资金 :{}'.format(Account.cash_available))

账户剩余资金 :960000.0


In [6]:
Account.hold

Series([], Name: amount, dtype: float64)

In [7]:
Account.init_hold

Series([], Name: amount, dtype: float64)

In [8]:
Account.hold_available

Series([], Name: amount, dtype: float64)

此时的账户cash并未减少,因为此过程为申报订单(已委托 未成交状态)

回测类接受订单 并返回订单(如果创建成功)

In [9]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

< QA_Order realorder_id Order_Pbp1zk4y datetime:2018-05-09 09:30:00 code:000001 amount:3600 price:10.955 towards:1 btype:stock_cn order_id:Order_Pbp1zk4y account:Acc_pGZ3xeaV status:queued >


In [10]:
#B.query_orders(rec_mes.account_cookie,rec_mes.realorder_id)

In [11]:
import pandas as pd

In [12]:
B.query_orders(Account.account_cookie)

,,order_time,code,name,towards,trade_price,order_price,status,order_amount,trade_amount,cancel_amount
account_cookie,realorder_id,,,,,,,,,,
Acc_pGZ3xeaV,Order_Pbp1zk4y,2018-05-09 09:31:00,000001,None,1,10.96,10.955,trade_success,3600,3600,0


In [13]:
#pd.DataFrame(list(B.deal_message.values()),columns=B.orderstatus_headers).set_index(['account_cookie','realorder_id'])

In [14]:
B.query_orders(Account.account_cookie,'filled')

,,trade_time,code,name,towards,trade_amount,trade_price,trade_money,trade_id
account_cookie,realorder_id,,,,,,,,
Acc_pGZ3xeaV,Order_Pbp1zk4y,2018-05-09 00:00:00,000001,None,1,3600,10.96,39447.8595,Trade_fHmrQ0OJ


In [15]:
trade_mes=B.query_orders(Account.account_cookie,'filled')
res=trade_mes.loc[Order.account_cookie,Order.realorder_id]
Order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)

In [16]:
#Order.trade((trade_id, trade_price, trade_amount, trade_time))

账户类接收到回测返回的回报信息,更新账户

In [17]:
Account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax,message,frozen
0,2018-05-09 00:00:00,000001,10.96,3600,960534.136,Order_Pbp1zk4y,Order_Pbp1zk4y,Trade_fHmrQ0OJ,Acc_pGZ3xeaV,9.864,0,None,0


此时我们可以打印一下现在的状态(现在的状态可以理解为在交易时 买入一只000001股票,但当天尚未收盘)

In [18]:
print('账户的可用资金 {}'.format(Account.cash_available))

账户的可用资金 960534.136


In [19]:
Account.hold

code
000001    3600
Name: amount, dtype: int64

In [20]:
Account.hold_available

code
000001    3600
Name: amount, dtype: int64

In [21]:
Account.init_hold.index_name='code'

In [22]:
import pandas as pd

In [23]:
pd.concat([Account.hold_available,Account.init_hold])

code
000001    3600
Name: amount, dtype: int64

In [24]:
+Account.hold_available

code
000001    3600
Name: amount, dtype: int64

我们注意到 当最初申报订单的时候,可用资金只有950.2999999999302元,而买入成功后,可用资金有3339.9289999998837元,原因是下单的时候模式是市价单模式(QA.ORDER_MODEL.MARKET),故实际成交金额为10.96元

买入以后 账户的持仓为90800股 000001

In [25]:
Account.hold

code
000001    3600
Name: amount, dtype: int64

买入后账户现金表被扩展

In [26]:
Account.cash

[1000000, 960534.136]

因为是t+1的A股市场,故此时可卖数量为0

In [27]:
Account.sell_available

Series([], Name: amount, dtype: float64)

# 执行结算

In [28]:
Account.settle()

# 结算后

In [29]:
Account.cash

[1000000, 960534.136]

In [30]:
Account.cash_available

960534.136

In [31]:
Account.sell_available

code
000001    3600
Name: amount, dtype: int64

In [32]:
Account.hold

code
000001    3600
Name: amount, dtype: int64

# 执行卖出操作

现在的持仓为: 000001 90800股

In [33]:
holdnum=Account.sell_available.get('000001',0)

In [34]:
holdnum

3600

申报一个卖出单,把可卖全部卖出

In [35]:
Order=Account.send_order(code='000001',
                        price=11,
                        amount=holdnum,
                        time='2018-05-10',
                        towards=QA.ORDER_DIRECTION.SELL,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                        )


In [36]:
Order

< QA_Order realorder_id Order_YGcx8UP5 datetime:2018-05-10 09:31:00 code:000001 amount:3600 price:11 towards:-1 btype:stock_cn order_id:Order_YGcx8UP5 account:Acc_pGZ3xeaV status:queued >

In [37]:
Account.cash_available  # 因为此时订单尚未申报成功 可用现金不变

960534.136

In [38]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

< QA_Order realorder_id Order_YGcx8UP5 datetime:2018-05-10 09:30:00 code:000001 amount:3600 price:11.0 towards:-1 btype:stock_cn order_id:Order_YGcx8UP5 account:Acc_pGZ3xeaV status:queued >


In [39]:
trade_mes=B.query_orders(Account.account_cookie,'filled')
res=trade_mes.loc[Order.account_cookie,Order.realorder_id]
Order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)

In [40]:
Account.cash_available # 此时订单已成交 cash_available立刻结转

1000084.636

In [41]:
Account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax,message,frozen
0,2018-05-09 00:00:00,000001,10.96,3600,960534.136,Order_Pbp1zk4y,Order_Pbp1zk4y,Trade_fHmrQ0OJ,Acc_pGZ3xeaV,9.864,0.0,None,0
1,2018-05-10 00:00:00,000001,11.00,-3600,1000084.636,Order_YGcx8UP5,Order_YGcx8UP5,Trade_6wnhQXyi,Acc_pGZ3xeaV,9.900,39.6,None,0


In [42]:
Account.orders

< QA_OrderQueue >

In [43]:
Account.orders.order_list

{'Order_Pbp1zk4y': < QA_Order realorder_id Order_Pbp1zk4y datetime:2018-05-09 09:30:00 code:000001 amount:3600 price:10.955 towards:1 btype:stock_cn order_id:Order_Pbp1zk4y account:Acc_pGZ3xeaV status:success_all >,
 'Order_YGcx8UP5': < QA_Order realorder_id Order_YGcx8UP5 datetime:2018-05-10 09:30:00 code:000001 amount:3600 price:11.0 towards:-1 btype:stock_cn order_id:Order_YGcx8UP5 account:Acc_pGZ3xeaV status:success_all >}

# 测试T0账户

In [44]:
 # 初始化一个account
    
AccountT0=portfolio.new_account(running_environment=QA.RUNNING_ENVIRONMENT.TZERO,init_hold={'000001':10000},init_cash=200000)

# 初始化一个回测类
B = QA.QA_BacktestBroker()

In [45]:
AccountT0.init_assets

{'cash': 200000, 'hold': {'000001': 10000}}

In [46]:
AccountT0.init_hold

code
000001    10000
Name: amount, dtype: int64

In [47]:
AccountT0.hold_available

Series([], Name: amount, dtype: float64)

In [48]:
AccountT0.sell_available

code
000001    10000
Name: amount, dtype: int64

这是一个有10000股000001持仓的T0账户

In [49]:
Order=AccountT0.send_order(code='000001',
                        price=11,
                        amount=AccountT0.sell_available.get('000001',0),
                        time='2018-05-10',
                        towards=QA.ORDER_DIRECTION.SELL,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                        )


In [50]:
Order.datetime

'2018-05-10 09:31:00'

In [51]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

< QA_Order realorder_id Order_FVBb5N7K datetime:2018-05-10 09:30:00 code:000001 amount:10000 price:11.0 towards:-1 btype:stock_cn order_id:Order_FVBb5N7K account:Acc_m7awZbrp status:queued >


In [52]:
rec_mes

< QA_Order realorder_id Order_FVBb5N7K datetime:2018-05-10 09:30:00 code:000001 amount:10000 price:11.0 towards:-1 btype:stock_cn order_id:Order_FVBb5N7K account:Acc_m7awZbrp status:queued >

In [53]:
B.query_orders(AccountT0.account_cookie,'filled')

,,trade_time,code,name,towards,trade_amount,trade_price,trade_money,trade_id
account_cookie,realorder_id,,,,,,,,
Acc_m7awZbrp,Order_FVBb5N7K,2018-05-10 00:00:00,000001,None,-1,10000,11.0,110137.5,Trade_NWdFv20n


In [54]:
trade_mes=B.query_orders(AccountT0.account_cookie,'filled')
res=trade_mes.loc[Order.account_cookie,Order.realorder_id]
Order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)

In [55]:
AccountT0.sell_available

code
000001   -10000
Name: amount, dtype: int64

In [56]:
Order.trade_time

['2018-05-10 00:00:00']

In [57]:
AccountT0.buy_available

code
000001   -10000
Name: amount, dtype: int64

In [58]:
AccountT0.hold_available

code
000001   -10000
Name: amount, dtype: int64

In [59]:
AccountT0.running_time

datetime.datetime(2019, 2, 5, 19, 47, 36, 73732)

In [60]:
AccountT0.datetime

'2018-05-10 00:00:00'

In [61]:
AccountT0.buy_available.get('000001')

-10000

In [62]:
r=AccountT0.close_positions_order

In [63]:
AccountT0.date

'2018-05-10'

In [64]:
for Order in r:
    #print(vars(Order))
    rec_mes=B.receive_order(QA.QA_Event(order=Order))
    trade_mes=B.query_orders(AccountT0.account_cookie,'filled')
    res=trade_mes.loc[Order.account_cookie,Order.realorder_id]
    Order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)

In [65]:
AccountT0.cash

[200000, 309862.5, 199734.975]

In [66]:
AccountT0.sell_available

code
000001   -20000
Name: amount, dtype: int64

In [67]:
AccountT0.hold_available

Series([], Name: amount, dtype: float64)

In [68]:
AccountT0.hold

code
000001    10000
Name: amount, dtype: int64

In [69]:
AccountT0.settle()

In [70]:
AccountT0.daily_hold

,,000001
date,account_cookie,
2018-05-10,Acc_m7awZbrp,0


In [71]:
AccountT0.daily_cash

,,date,cash,datetime,account_cookie
date,account_cookie,,,,
2018-05-10,Acc_m7awZbrp,2018-05-10,199734.975,2018-05-10 00:00:00,Acc_m7awZbrp


In [72]:
AccountT0.hold_table()

code
000001    10000
Name: amount, dtype: int64

In [73]:
AccountT0.hold_price()

Series([], dtype: float64)

In [74]:
AccountT0.datetime

'2018-05-11 09:30:00'

In [75]:
AccountT0.sell_available

code
000001    10000
Name: amount, dtype: int64

In [76]:
risk_t0=QA.QA_Risk(AccountT0)

In [77]:
risk_t0.init_assets

{'cash': 200000, 'hold': {'000001': 10000}}

In [78]:
risk_t0.init_cash-risk_t0.assets

date
2018-05-10    265.025
Name: 0, dtype: float64

In [79]:
risk_t0.assets.iloc[-1]/risk_t0.init_cash-1

-0.001325124999999927

In [80]:
risk_t0.profit_construct

{'total_buyandsell': -99.99,
 'total_tax': -110.0,
 'total_commission': -55.03,
 'total_profit': -265.02}

# 测试期货账户

In [81]:

rb_ds=QA.QA_fetch_future_min_adv('RBL8','2019-01-01','2019-02-01','15min')

AccountFuture=portfolio.new_account(init_cash=1000000,allow_sellopen=True,allow_t0=True,account_cookie='future_test',market_type=QA.MARKET_TYPE.FUTURE_CN,frequence=QA.FREQUENCE.FIFTEEN_MIN)


#Account.reset_assets(10000000)
Broker=QA.QA_BacktestBroker()


In [82]:
#rb_ds.data

In [83]:
# 全仓买入'000001'
hq=next(rb_ds.panel_gen)
order=AccountFuture.send_order(code='RBL8',
                        price=hq.open,
                        money=AccountFuture.cash_available,
                        time=hq.datetime[0],
                        towards=QA.ORDER_DIRECTION.BUY_OPEN,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_MONEY
                        )


In [84]:
Broker.receive_order(QA.QA_Event(order=order,market_data=hq))

< QA_Order realorder_id Order_2CQtTA8q datetime:2019-01-02 09:15:00 code:RBL8 amount:200 price:3409.5 towards:2 btype:future_cn order_id:Order_2CQtTA8q account:future_test status:queued >

In [85]:
trade_mes = Broker.query_orders(AccountFuture.account_cookie, 'filled')
res = trade_mes.loc[order.account_cookie, order.realorder_id]
order.trade(res.trade_id, res.trade_price, res.trade_amount, res.trade_time)

In [87]:
AccountFuture.save()

account a1 save
account a2 save
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
account Acc_pGZ3xeaV save
account Acc_m7awZbrp save
account future_test save
